# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras import initializers

# import keras
# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense, Embedding
# from keras.layers import SimpleRNN
# from keras.datasets import imdb
# from keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
c:\users\moon\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


c:\users\moon\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [4]:
len(x_train[10093])

277

In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
y_train.shape

(25000,)

In [7]:
x_train[-1]

array([  27,  239,    9,   43, 8368,  209,  405,   10,   10,   12,  764,
         40,    4,  248,   20,   12,   16,    5,  174, 1791,   72,    7,
         51,    6, 1739,   22,    4,  204,  131,    9])

In [8]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [12]:
x_train.shape[1:]

(30,)

In [13]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
x_train.shape[1:]

(30,)

In [15]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 280       
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [17]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [18]:
import datetime

now = datetime.datetime.now

t = now()
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))
print("Training time: {}s".format(now() - t))

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.6731 - accuracy: 0.6072 - val_loss: 0.6302 - val_accuracy: 0.6627
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.5825 - accuracy: 0.6978 - val_loss: 0.5667 - val_accuracy: 0.7028
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.5239 - accuracy: 0.7427 - val_loss: 0.5362 - val_accuracy: 0.7254
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4864 - accuracy: 0.7681 - val_loss: 0.5169 - val_accuracy: 0.7392
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4571 - accuracy: 0.7872 - val_loss: 0.4963 - val_accuracy: 0.7546
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4341 - accuracy: 0.8028 - val_loss: 0.4835 - val_accuracy: 0.7616
Epoch 7/10
782/782 [==============================] - 7s 8ms/step - loss: 0.4150 - accuracy: 0.8144 - val_loss: 0.4730 - val_accuracy: 0.7680
Epoch 

In [19]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 1s 2ms/step - loss: 0.4548 - accuracy: 0.7819
Test score: 0.45477330684661865
Test accuracy: 0.7818800210952759


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [20]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [30]:
rnn_hidden_dim = 5
word_embedding_dim = 80
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [31]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [37]:
t = now()
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))
print('Training time: {}s'.format(now() - t))

Epoch 1/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4392 - accuracy: 0.8434 - val_loss: 0.4894 - val_accuracy: 0.7920
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4296 - accuracy: 0.8465 - val_loss: 0.4776 - val_accuracy: 0.8048
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4207 - accuracy: 0.8494 - val_loss: 0.4739 - val_accuracy: 0.8065
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4124 - accuracy: 0.8492 - val_loss: 0.4612 - val_accuracy: 0.8097
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4049 - accuracy: 0.8519 - val_loss: 0.4721 - val_accuracy: 0.8052
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3976 - accuracy: 0.8563 - val_loss: 0.4545 - val_accuracy: 0.8097
Epoch 7/10
782/782 [==============================] - 9s 12ms/step - loss: 0.3905 - accuracy: 0.8561 - val_loss: 0.5161 - val_accuracy: 

In [29]:
x_train.shape

(25000, 80)

In [32]:
model_rnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 80)          1600000   
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 5)                 430       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 1,600,436
Trainable params: 1,600,436
Non-trainable params: 0
_________________________________________________________________


In [33]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [34]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [35]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [36]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 11s 13ms/step - loss: 0.6601 - accuracy: 0.5920 - val_loss: 0.6200 - val_accuracy: 0.7367
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5935 - accuracy: 0.7489 - val_loss: 0.5842 - val_accuracy: 0.7582
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5595 - accuracy: 0.7898 - val_loss: 0.5635 - val_accuracy: 0.7897
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5373 - accuracy: 0.8080 - val_loss: 0.5800 - val_accuracy: 0.7446
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5198 - accuracy: 0.8186 - val_loss: 0.5489 - val_accuracy: 0.7884
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5033 - accuracy: 0.8261 - val_loss: 0.5265 - val_accuracy: 0.8026
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4877 - accuracy: 0.8312 - val_loss: 0.5110 - val_accuracy:

In [38]:
# Out of curiosity, run for 10 more epochs
import datetime 

now = datetime.datetime.now

t = now()
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))
print('Training time: %s' % (now() - t))
print('Training time: {}s'.format(now() - t))

Epoch 1/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3650 - accuracy: 0.8644 - val_loss: 0.4312 - val_accuracy: 0.8169
Epoch 2/10
141/782 [====>.........................] - ETA: 6s - loss: 0.3632 - accuracy: 0.8659

KeyboardInterrupt: 

---
### Machine Learning Foundation (C) 2020 IBM Corporation